In [1]:
# train model 1
from tensorflow.keras.applications import Xception
from tensorflow.keras import layers, models

base_model = Xception(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(2, activation='softmax')
])

2025-07-08 11:43:31.247722: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [5]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
from module.dataloader import create_generators

data_dir = "data/"
train_gen, val_gen = create_generators(
    data_dir,
    img_size=(224, 224),   
    batch_size=16,
    val_split=0.2,
    augment=False
)

Found 32000 images belonging to 2 classes.
Found 8000 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

callbacks = [
    EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)
]

history = model.fit(
    train_gen,
    epochs=3,
    validation_data=val_gen,
    callbacks=callbacks
)

Epoch 1/3
  58/2000 ━━━━━━━━━━━━━━━━━━━━ 32:54 1s/step - accuracy: 0.9980 - loss: 0.0145

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.legend()
plt.title('Accuracy: VGG19')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.title('Loss: VGG19')

plt.show()


In [2]:
model.save("models/xceptionmodel.keras")